## Deliverable 2. Create a Customer Travel Destinations Map.

In [12]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import csv

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [15]:
# 1. Import the WeatherPy_database.csv file. 
weather_data_to_load = os.path.join("..", "Weather_Database", "WeatherPy_Database.csv")

city_data_df = pd.read_csv(weather_data_to_load)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cherskiy,RU,68.7500,161.3000,-1.39,99,92,5.32,overcast clouds
1,1,Coquimbo,CL,-29.9533,-71.3436,55.13,87,0,3.44,clear sky
2,2,Tuktoyaktuk,CA,69.4541,-133.0374,21.20,79,90,4.61,light snow
3,3,Te Anau,NZ,-45.4167,167.7167,66.29,52,89,4.27,overcast clouds
4,4,Ribeira Grande,PT,38.5167,-28.7000,69.66,85,60,22.28,broken clouds


In [16]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 100


In [17]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Te Anau,NZ,-45.4167,167.7167,66.29,52,89,4.27,overcast clouds
4,4,Ribeira Grande,PT,38.5167,-28.7000,69.66,85,60,22.28,broken clouds
8,8,Vaini,TO,-21.2000,-175.2000,77.16,100,90,5.75,moderate rain
10,10,Katsuura,JP,35.1333,140.3000,69.82,57,62,15.99,broken clouds
12,12,Rikitea,PF,-23.1203,-134.9692,75.51,77,100,8.93,light rain
13,13,Katobu,ID,-4.9408,122.5276,90.19,58,100,2.04,overcast clouds
18,18,West Bay,KY,19.3667,-81.4167,79.07,69,99,1.01,overcast clouds
19,19,Yei,SS,4.0950,30.6779,69.57,89,36,3.78,scattered clouds
27,27,Mani,CO,4.8172,-72.2886,75.85,82,100,4.03,overcast clouds
30,30,Busselton,AU,-33.6500,115.3333,71.10,46,13,7.85,few clouds


In [22]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                306
City                   306
Country                304
Lat                    306
Lng                    306
Max Temp               306
Humidity               306
Cloudiness             306
Wind Speed             306
Current Description    306
dtype: int64

In [23]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Te Anau,NZ,-45.4167,167.7167,66.29,52,89,4.27,overcast clouds
4,4,Ribeira Grande,PT,38.5167,-28.7000,69.66,85,60,22.28,broken clouds
8,8,Vaini,TO,-21.2000,-175.2000,77.16,100,90,5.75,moderate rain
10,10,Katsuura,JP,35.1333,140.3000,69.82,57,62,15.99,broken clouds
12,12,Rikitea,PF,-23.1203,-134.9692,75.51,77,100,8.93,light rain
...,...,...,...,...,...,...,...,...,...,...
661,661,Limbdi,IN,22.5667,71.8000,84.94,22,47,13.22,scattered clouds
663,663,Biak,ID,-0.9131,122.8766,83.80,74,85,2.46,overcast clouds
664,664,Port Moresby,PG,-9.4431,147.1797,90.82,62,20,17.27,haze
665,665,Kosai,JP,34.7105,137.5316,74.93,51,36,24.70,scattered clouds


In [24]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Te Anau,NZ,66.29,overcast clouds,-45.4167,167.7167,
4,Ribeira Grande,PT,69.66,broken clouds,38.5167,-28.7000,
8,Vaini,TO,77.16,moderate rain,-21.2000,-175.2000,
10,Katsuura,JP,69.82,broken clouds,35.1333,140.3000,
12,Rikitea,PF,75.51,light rain,-23.1203,-134.9692,
13,Katobu,ID,90.19,overcast clouds,-4.9408,122.5276,
18,West Bay,KY,79.07,overcast clouds,19.3667,-81.4167,
19,Yei,SS,69.57,scattered clouds,4.0950,30.6779,
27,Mani,CO,75.85,overcast clouds,4.8172,-72.2886,
30,Busselton,AU,71.10,few clouds,-33.6500,115.3333,


In [25]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [26]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Te Anau,NZ,66.29,overcast clouds,-45.4167,167.7167,Kingsgate Hotel Te Anau
4,Ribeira Grande,PT,69.66,broken clouds,38.5167,-28.7000,Quinta da Meia Eira
8,Vaini,TO,77.16,moderate rain,-21.2000,-175.2000,Keleti Beach Resort
10,Katsuura,JP,69.82,broken clouds,35.1333,140.3000,Katsuura Hotel Mikazuki
12,Rikitea,PF,75.51,light rain,-23.1203,-134.9692,People ThankYou


In [34]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df['Hotel Name']!='')]

clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Te Anau,NZ,66.29,overcast clouds,-45.4167,167.7167,Kingsgate Hotel Te Anau
4,Ribeira Grande,PT,69.66,broken clouds,38.5167,-28.7000,Quinta da Meia Eira
8,Vaini,TO,77.16,moderate rain,-21.2000,-175.2000,Keleti Beach Resort
10,Katsuura,JP,69.82,broken clouds,35.1333,140.3000,Katsuura Hotel Mikazuki
12,Rikitea,PF,75.51,light rain,-23.1203,-134.9692,People ThankYou
...,...,...,...,...,...,...,...
661,Limbdi,IN,84.94,scattered clouds,22.5667,71.8000,LIMBDI YOGI HOTEL
663,Biak,ID,83.80,overcast clouds,-0.9131,122.8766,Penginapan Kim
664,Port Moresby,PG,90.82,haze,-9.4431,147.1797,Lamana Hotel (Port Moresby)
665,Kosai,JP,74.93,scattered clouds,34.7105,137.5316,Hotel Route Inn Hamanako


In [42]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [43]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [44]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))